# Supabase Database Setup

This notebook creates the tables for the AskSpatz negotiation system.

In [2]:
# Install dependencies if needed
!pip install supabase python-dotenv

  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached StrEnum-0.4.15-py3-none-any.whl.metadata (5.3 kB)
  Using cached h2-4.3.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
Using cached h2-4.3.0-py3-none-any.whl (61 kB)
Using cached hpack-4.1.0-py3-none-any.whl (34 kB)
Using cached hyperframe-6.1.0-py3-none-any.whl (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 191.2 kB/s eta 0:00:00a 0:00:01
Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
Using cached PyJWT-2.10.1-py3-none-any.whl (22 kB)
Using cached StrEnum-0.4.15-py3-none-any.whl (8.9 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
  Attempting uninstall: typing-extensions
    Found existing installat

In [4]:
import os
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()

# Get Supabase credentials from environment
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")  # Use service_role key for admin operations

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("Missing SUPABASE_URL or SUPABASE_KEY in .env file")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
print("Connected to Supabase!")

Connected to Supabase!


## SQL to Create Tables

Run this SQL in the Supabase SQL Editor (Dashboard > SQL Editor), or execute via the Python client below.

In [5]:
create_tables_sql = """
-- Vendors table
CREATE TABLE IF NOT EXISTS vendors (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    behaviour TEXT
);
-- Products table
CREATE TABLE IF NOT EXISTS products (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    vendor_ids INTEGER[] DEFAULT '{}'
);

-- Negotiation Group table
CREATE TABLE IF NOT EXISTS negotiation_group (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    product INTEGER REFERENCES products(id) ON DELETE SET NULL,
    quantity INTEGER DEFAULT 1
);

-- Negotiation table
CREATE TABLE IF NOT EXISTS negotiation (
    id SERIAL PRIMARY KEY,
    negotiation_group_id INTEGER REFERENCES negotiation_group(id) ON DELETE CASCADE,
    conversation_id INTEGER,
    vendor_id INTEGER REFERENCES vendors(id) ON DELETE SET NULL
);

-- Message table
CREATE TABLE IF NOT EXISTS message (
    id SERIAL PRIMARY KEY,
    type TEXT,
    message TEXT,
    conversation_id INTEGER,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

-- Offer table
CREATE TABLE IF NOT EXISTS offer (
    id SERIAL PRIMARY KEY,
    negotiation_id INTEGER REFERENCES negotiation(id) ON DELETE CASCADE,
    description TEXT,
    price DECIMAL(12, 2)
);
"""

print(create_tables_sql)


-- Vendors table
CREATE TABLE IF NOT EXISTS vendors (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    behaviour TEXT
);
-- Products table
CREATE TABLE IF NOT EXISTS products (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    vendor_ids INTEGER[] DEFAULT '{}'
);

-- Negotiation Group table
CREATE TABLE IF NOT EXISTS negotiation_group (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    product INTEGER REFERENCES products(id) ON DELETE SET NULL,
    quantity INTEGER DEFAULT 1
);

-- Negotiation table
CREATE TABLE IF NOT EXISTS negotiation (
    id SERIAL PRIMARY KEY,
    negotiation_group_id INTEGER REFERENCES negotiation_group(id) ON DELETE CASCADE,
    conversation_id INTEGER,
    vendor_id INTEGER REFERENCES vendors(id) ON DELETE SET NULL
);

-- Message table
CREATE TABLE IF NOT EXISTS message (
    id SERIAL PRIMARY KEY,
    type TEXT,
    message TEXT,
    conversation_id INTEGER,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

-- Offer table
CREATE TABLE IF NOT EXIST

In [6]:
# Execute the SQL using Supabase RPC or run it manually in the SQL Editor
# Note: You may need to run this directly in the Supabase SQL Editor
# as the Python client doesn't support raw DDL commands directly.

# Option 1: Copy the SQL above and paste it into Supabase Dashboard > SQL Editor

# Option 2: If you have a Postgres connection string, use psycopg2:
# import psycopg2
# conn = psycopg2.connect(os.getenv("DATABASE_URL"))
# cur = conn.cursor()
# cur.execute(create_tables_sql)
# conn.commit()
# cur.close()
# conn.close()

print("Copy the SQL above and run it in the Supabase SQL Editor!")

Copy the SQL above and run it in the Supabase SQL Editor!


## Verify Tables Were Created

In [ ]:
# Test that tables exist by querying them
tables = ['vendors', 'products', 'negotiation_group', 'negotiation', 'message', 'offer']

for table in tables:
    try:
        result = supabase.table(table).select('*').limit(1).execute()
        print(f"✓ Table '{table}' exists")
    except Exception as e:
        print(f"✗ Table '{table}' not found or error: {e}")